In [11]:
import json
import re

def text2json(text):
    # Regular expression to find JSON within text
    json_regex = re.compile(r'```json(.*?)```', re.DOTALL)
    json_match = json_regex.search(text)
    
    if json_match:
        json_text = json_match.group(1).strip()
    else:
        # Attempt to extract JSON without delimiters
        try:
            return json.loads(text)
        except json.JSONDecodeError:
            return {"content": text.strip()}
    
    # Attempt to parse JSON
    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        return {"content": text.strip()}

In [10]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

def openai_prompt(settings, system, user):
    # Set default settings if not provided
    default_settings = {"model": "gpt-3.5-turbo"}
    local_settings = {**default_settings, **settings}

    # Concatenate the messages
    system_objects = [{"role": "system", "content": item} for item in system]
    user_objects = [{"role": "user", "content": item} for item in user]

    # Prepare the API call parameters
    api_params = {
        "model": local_settings["model"],
        "messages": system_objects + user_objects
    }

    # Include max_tokens if present in settings
    if "max_tokens" in local_settings:
        api_params["max_tokens"] = local_settings["max_tokens"]
    
    # Create the OpenAI chat completion
    response = openai.chat.completions.create(**api_params)
    #print("RESPONSE")
    #print(response)
    content = response.choices[0].message.content
    #print("CONTENT")
    #print(content)
    data = text2json(content)
    return data